In [ ]:
import sqlite3
import pandas as pd
import gradio as gr
from itertools import permutations as perm

In [ ]:
def decompose(n: int):
    n_ = [i+1 for i in range(n)]
    n_ = [i/n for i in n_]
    n_ = [i/sum(n_) for i in n_]
    return n_

In [ ]:
con = sqlite3.connect('database.sqlite')

In [ ]:
country = pd.read_sql('SELECT * FROM Country', con)
country.head()

In [ ]:
league = pd.read_sql('SELECT * FROM League', con)
league.head()

In [ ]:
match = pd.read_sql('SELECT * FROM Match', con)
match.head()

In [ ]:
player = pd.read_sql('SELECT * FROM Player', con)
player.head()

In [ ]:
player_att = pd.read_sql('SELECT * FROM Player_Attributes', con)
player_att.head()

In [ ]:
team = pd.read_sql('SELECT * FROM Team', con)
team.head()

In [ ]:
team_att = pd.read_sql('SELECT * FROM Team_Attributes', con)
team_att.head()

#### We will use 4 datasets:
1. country - there are 11 countries
2. league - there are 11; one per country
3. match - there are 25,979 matches for 8 seasons spanning 2008/2009 to 2015/2016
4. team - there are 299 teams

### THE PLAN
#### 1. Extract on the columns you'll need for each table
* country, league, and team are fine
* for match, pick only the needed onesm_goal',

In [ ]:
match = pd.read_sql('SELECT id, country_id, league_id, season, match_api_id, home_team_api_id, away_team_api_id, home_team_goal, away_team_goal FROM Match', con)

In [ ]:
home_matches = team.merge(match, how='right', left_on='team_api_id', right_on='home_team_api_id').drop(columns=['id_x', 'id_y'])
home_matches = home_matches.merge(country, how='left', left_on='country_id', right_on='id').drop(columns=['id'])
home_matches = home_matches.merge(league, how='left', left_on='league_id', right_on='id').drop(columns=['id'])
home_matches.head()

In [ ]:
#Clean up colum names
home_matches.drop(columns='country_id_y', inplace=True)
home_matches.head()

In [ ]:
home_matches.rename(columns={'country_id_x':'country_id', 'name_x':'country', 'name_y':'league'}, inplace=True)
home_matches.head()

In [ ]:
home_matches['home_win'] = home_matches['home_team_goal'] > home_matches['away_team_goal']
home_matches['home_loss'] = home_matches['home_team_goal'] < home_matches['away_team_goal']
home_matches['home_draw'] = home_matches['home_team_goal'] == home_matches['away_team_goal']
home_matches.head()

In [ ]:
seasons_list = sorted(match['season'].unique())
seasons_list

In [ ]:
n=len(seasons_list)
seasons = pd.DataFrame(seasons_list, columns=['season'])
seasons['weight'] = decompose(n)
seasons

In [ ]:
home_matches = home_matches.merge(seasons, on='season')
home_matches.head()

In [ ]:
home_matches['home_win_odds'] = home_matches['home_win']*home_matches['weight']
home_matches['home_loss_odds'] = home_matches['home_loss']*home_matches['weight']
home_matches['home_draw_odds'] = home_matches['home_draw']*home_matches['weight']
home_matches.head()

In [ ]:
home_odds = home_matches.groupby(['team_api_id', 'team_long_name', 'team_short_name', 'country_id', 'league_id', 'country', 'league']).sum()
home_odds.reset_index(inplace=True)
home_odds.drop(columns=['season', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'weight'], inplace=True)
home_odds.head()

In [ ]:
assert(team['id'].shape[0] == home_odds['team_api_id'].shape[0])

# AWAY ANALYSIS

In [ ]:
away_matches = team.merge(match, how='right', left_on='team_api_id', right_on='away_team_api_id').drop(columns=['id_x', 'id_y'])
away_matches = away_matches.merge(country, how='left', left_on='country_id', right_on='id').drop(columns=['id'])
away_matches = away_matches.merge(league, how='left', left_on='league_id', right_on='id').drop(columns=['id'])

away_matches.drop(columns='country_id_y', inplace=True)
away_matches.head()
away_matches.rename(columns={'country_id_x':'country_id', 'name_x':'country', 'name_y':'league'}, inplace=True)

away_matches['away_win'] = away_matches['away_team_goal'] > away_matches['home_team_goal']
away_matches['away_loss'] = away_matches['away_team_goal'] < away_matches['home_team_goal']
away_matches['away_draw'] = away_matches['away_team_goal'] == away_matches['home_team_goal']

away_matches = away_matches.merge(seasons, on='season')

away_matches['away_win_odds'] = away_matches['away_win']*away_matches['weight']
away_matches['away_loss_odds'] = away_matches['away_loss']*away_matches['weight']
away_matches['away_draw_odds'] = away_matches['away_draw']*away_matches['weight']

away_odds = away_matches.groupby(['team_api_id', 'team_long_name', 'team_short_name', 'country_id', 'league_id', 'country', 'league']).sum()
away_odds.reset_index(inplace=True)
away_odds.drop(columns=['season', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'weight'], inplace=True)

away_odds.head()

In [ ]:
assert(team['id'].shape[0] == away_odds['team_api_id'].shape[0])

In [ ]:
def predict_result(home, away): # Takes in team id as integer or team short name as string
    
    if (type(home) == str and str.upper(home) not in list(team['team_short_name'])) or (type(away) == str and str.upper(away) not in list(team['team_short_name'])):
        return 'ERROR: Team(s) not found!'

    if type(home) == str:
        home = team[team['team_short_name'] == str.upper(home)].reset_index()['team_api_id'][0]

    if type(away) == str:
        away = team[team['team_short_name'] == str.upper(away)].reset_index()['team_api_id'][0]
        
    if home not in list(team['team_api_id']) or away not in list(team['team_api_id']):
        return 'ERROR: Team(s) not found!'
        
    if home == away:
        return 'ERROR: Home team cannot be the same as away team!'
        
    else:
        
        home_team_name = team[team['team_api_id']==home].reset_index()['team_long_name'][0]
        away_team_name = team[team['team_api_id']==away].reset_index()['team_long_name'][0]
    
        home_team_odds = home_odds[home_odds['team_api_id']==home][['team_api_id', 'home_win_odds', 'home_loss_odds', 'home_draw_odds']].reset_index()
        away_team_odds = away_odds[away_odds['team_api_id']==away][['team_api_id', 'away_win_odds', 'away_loss_odds', 'away_draw_odds']].reset_index()
        
        home_win_odd = home_team_odds['home_win_odds'] * away_team_odds['away_loss_odds']
        away_win_odd = home_team_odds['home_loss_odds'] * away_team_odds['away_win_odds']
        draw_odd = home_team_odds['home_draw_odds'] * away_team_odds['away_draw_odds']
        total_odds = home_win_odd + away_win_odd + draw_odd
        
        home_win_prob = (home_win_odd/total_odds)[0]
        away_win_prob = (away_win_odd/total_odds)[0]
        draw_prob = (draw_odd/total_odds)[0]
    
        return f'MATCH OUTCOME PROBABILITY\n{home_team_name} : {home_win_prob:0.0%}\nDraw: {draw_prob:0.0%}\n{away_team_name} : {away_win_prob:0.0%}'


In [ ]:
log = pd.DataFrame(perm(sorted(team['team_short_name'].unique().tolist()), 2), columns=['home', 'away'])
log.to_csv('log/log.csv', index=False)

In [ ]:
interface = gr.Interface(fn=predict_result,
                         inputs=['text', 'text'],
                         outputs='text',
                         #examples='log/',
                         #flagging_mode='manual',
                         #flagging_dir='log/',
                         title='Predict football match outcomes in one second!!!',
                         description='Input the home and away teams in short form.\n\nE.g. ARS and AVL',
                        max_batch_size=20)
interface.launch()